In [1]:
from datetime import datetime, timedelta
import os

import pandas as pd

In [3]:
def get_adjusted_date():
    'Treat weekend dates as preceding Friday'
    NOW = datetime.now()
    weekday = NOW.weekday()
    if weekday == 5:    # Sat                                                   
        return NOW - timedelta(1)
    elif weekday == 6:  # Sun                                                   
        return NOW - timedelta(2)
    return NOW

In [5]:
HOME = os.environ['HOME']
DOWNLOADS = f'{HOME}/Downloads'
NOW = get_adjusted_date()
TODAY = NOW.date()
TOMORROW = (NOW + timedelta(1)).date()

In [6]:
def preclean_etrade(path):
    print(f'Pre-cleaning {path}...')
    with open(path, 'r') as fi:
        data_in = fi.readlines()
        with open(path, 'w') as fo:
            for line in data_in:
                fields = line.split(',')
                if len(fields) > 2 and fields[2] == '"--"':
                    continue
                else:
                    fo.write(line)

In [34]:
def upload_etrade():
    filename = 'Positions.csv'
    path = f'{DOWNLOADS}/{filename}'
    preclean_etrade(path)
    print('Uploading E*Trade data...')
    etrade = (
        pd
        .read_csv(path, index_col=0, skiprows=1)[['Market Value']]
        .rename(columns={'Market Value': 'et'}))
    etrade.index = map(lambda x: x.split()[0], etrade.index)
    drops = [d for d in ['FNRG', 'Portfolio', 'Cash'] if d in etrade.index]
    etrade.drop(index=drops, inplace=True)
    if len(etrade[etrade.et == '--']):
        raise ValueError(
            f'Missing price data in ETrade file ({filename}). Correct.')
    return etrade

In [35]:
etrade = upload_etrade()
etrade.head()

Pre-cleaning /Users/damiansp/Downloads/Positions.csv...
Uploading E*Trade data...


,et
AMD,4722.00
AMGN,3779.72
ANET,264.70
AXON,308.23
BYND,294.40


In [11]:
def separate_accounts(fid):
    accounts = {
        account_name: fid[fid['Account Name'] == account_name]
        for account_name in fid['Account Name'].unique()}
    for acct in accounts:
        accounts[acct].index = accounts[acct].Symbol
        accounts[acct] = (
            accounts[acct]
            .rename(columns={'Current Value': acct})
            .drop(columns=['Account Name', 'Symbol']))
    return accounts

In [14]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [45]:
def upload_fidelity():
    print('Uploading Fidelity data...')
    today = datetime.strftime(TODAY, '%b-%d-%Y')
    filename = f'Portfolio_Positions_{today}.csv'
    print('Looking for Fidelity file:', filename)
    try:
        fidelity = pd.read_csv(f'{DOWNLOADS}/{filename}')
    except FileNotFoundError:
        tomorrow = datetime.strftime(TOMORROW, '%b-%d-%Y')
        filename = filename.replace(today, tomorrow)
        fidelity = pd.read_csv(f'{DOWNLOADS}/{filename}')
    fidelity = (
        fidelity[['Account Name', 'Symbol', 'Current Value']].dropna())
    fidelity['Current Value'] = (
        fidelity['Current Value']
        .apply(convert_value)
        .fillna(0)
        .astype(int))
    fidelity = fidelity[
        ((fidelity.Symbol != 'SPAXX**')
         & (fidelity.Symbol != 'Pending Activity'))]
    fidelity = separate_accounts(fidelity)
    fidelity = pd.concat([v for v in fidelity.values()], axis=1)
    fidelity.columns = ['rollover', 'roth', 'simple']
    fidelity['fid'] = fidelity.sum(axis=1)
    return fidelity

In [46]:
fidelity = upload_fidelity()
fidelity.head()

Uploading Fidelity data...
Looking for Fidelity file: Portfolio_Positions_Apr-26-2024.csv


,rollover,roth,simple,fid
Symbol,,,,
KNSL,749.0,4870.0,NaN,5619.0
MCD,3004.0,4096.0,NaN,7100.0
JNJ,5261.0,NaN,NaN,5261.0
ROKU,113.0,3663.0,NaN,3776.0
QDEL,3805.0,1427.0,NaN,5232.0


In [47]:
def parse_schwab(path):
    data = []
    inds = []
    is_header = True
    with open(path, 'r') as f:
        for line in f:
            if line.startswith('"Cash') or line.startswith('"Account'):
                continue
            if not is_header:
                cols = line.split(',"')
                symbol = cols[0].strip('""')
                amt = float(cols[6].strip('"$'))
                inds.append(symbol)
                data.append(amt)
            if line.startswith('"Symbol"'):
                is_header = False
    return pd.DataFrame({'schwab': data}, index=inds)

In [48]:
def upload_schwab():
    print('Uploading Schwab data...')
    path_start = f'PCRA_Custodial-Positions-{str(TODAY)}'
    print(f'Looking for Schwab file: {path_start}...')
    filename = [
        f for f in os.listdir(DOWNLOADS) if f.startswith(path_start)
    ][0]
    path = f'{DOWNLOADS}/{filename}'
    schwab = parse_schwab(path)
    return schwab

In [49]:
schwab = upload_schwab()
schwab.head()

Uploading Schwab data...
Looking for Schwab file: PCRA_Custodial-Positions-2024-04-26...


,schwab
AAPL,169.30
ABBV,159.62
ADSK,217.93
ADUS,94.54
AMBA,129.27


In [50]:
def upload_dm():
    dm = pd.read_csv(
        f'{DOWNLOADS}/Dongmei.csv',
        index_col=0,
        usecols=['Position', 'Market Value']
    ).rename(columns={'Market Value': 'dm'})
    dm.index = [x.replace(' shares', '') for x in dm.index]
    return dm

In [51]:
dm = upload_dm()
dm.head()

,dm
ADYEY,229.60
AMZN,180.00
ASML,953.41
BYND,37.45
CHWY,376.74


In [52]:
def upload_sims():
    print('Uploading simulation data...')
    #files = [f for f in os.listdir(DOWNLOADS) if f.startswith('Holdings')]
    files = [
        f'Holdings - Damian Satterthwaite-Phillips{x}.csv' for
        x in ['', '(1)', '(2)']]
    print(f'Found {len(files)} sim files.')
    dfs = []
    for f in files:
        df = pd.read_csv(
            f'{DOWNLOADS}/{f}', index_col=0, usecols=['Symbol', 'Value'])
        df.Value = df.Value.str.replace(',', '').str[1:].astype(float)
        dfs.append(df)
    out = pd.concat(dfs, axis=1)
    out.columns = [f'sim{i}' for i in range(1, len(files) + 1)]
    return out

In [53]:
sims = upload_sims()
sims.head()

Uploading simulation data...
Found 3 sim files.


,sim1,sim2,sim3
Symbol,,,
AAPL,338.60,846.50,338.60
ACN,5236.17,12012.39,7392.24
ADBE,2387.80,2387.80,8118.52
ADSK,3268.95,6319.97,435.86
AMBA,1034.16,86.18,2154.50


In [66]:
def load():
    etrade = upload_etrade()
    fidelity = upload_fidelity()
    schwab = upload_schwab()
    dm = upload_dm()
    sims = upload_sims()
    out = pd.concat(
        [etrade, fidelity, schwab, dm, sims], axis=1
    ).sort_index()
    out = out.astype(float).fillna(0)
    out['owned'] = out.et + out.fid + out.schwab
    out = out.round().astype(int)
    return out

In [67]:
out = load()
out.head()

Pre-cleaning /Users/damiansp/Downloads/Positions.csv...
Uploading E*Trade data...
Uploading Fidelity data...
Looking for Fidelity file: Portfolio_Positions_Apr-26-2024.csv
Uploading Schwab data...
Looking for Schwab file: PCRA_Custodial-Positions-2024-04-26...
Uploading simulation data...
Found 3 sim files.


,et,rollover,roth,simple,fid,schwab,dm,sim1,sim2,sim3,owned
AAPL,0,0,4402,0,4402,169,0,339,846,339,4571
ABBV,0,0,0,0,0,160,0,0,0,0,160
ABNB,0,0,328,0,328,0,0,0,0,0,328
ACN,0,0,6468,0,6468,0,0,5236,12012,7392,6468
ADBE,0,0,6686,0,6686,0,0,2388,2388,8119,6686
